In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [12]:
train_csv = pd.read_csv('./dataset/train.csv')

In [14]:
from sklearn.preprocessing import OneHotEncoder
one_hot = OneHotEncoder()
label = one_hot.fit_transform(np.array([train_csv['label'].values]).T).toarray()

In [4]:
train_df = train_csv.drop(['label'], axis=1)

In [97]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_df, label, test_size=0.1, random_state=42)

In [6]:
x = tf.placeholder(tf.float32, [None, 784])
w = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

y = tf.nn.softmax(tf.matmul(x, w) + b)

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
y_ = tf.placeholder("float", [None,10])

In [8]:
corss_entropy = -tf.reduce_sum(y_*tf.log(y))
learning_rate = 0.01
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(corss_entropy)

In [83]:
class Dataset(object):
    def __init__(self, data, label):
        self.rows = len(data.values)
        self.images = data.values
        self.images = self.images.astype(np.float32)
        self.images = np.multiply(self.images, 1.0 / 255.0)
        self.labels = label
        self.index_in_epoch = 0
        self.epoch = 0
    def next_batch(self, batch_size):
        start = self.index_in_epoch
        self.index_in_epoch += batch_size
        if self.index_in_epoch > self.rows:
            self.epoch += 1
            perm = np.arange(self.rows)
            np.random.shuffle(perm)
            self.images = self.images[perm]
            self.labels = self.labels[perm]
            #next epoch
            start = 0
            self.index_in_epoch = batch_size
        end = self.index_in_epoch
        return self.images[start:end] , self.labels[start:end]

In [98]:
dataset = Dataset(X_train, y_train)

In [99]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for i in range(1000):
    X_train_, y_train_ = dataset.next_batch(100)
    sess.run(train_step, feed_dict={x: X_train_, y_: y_train_})

In [100]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
print(sess.run(accuracy, feed_dict={x: X_test, y_: y_test}))

0.8442857


In [104]:
sess.run(init)

dataset = Dataset(train_df, label)
for i in range(1000):
    X_train_, y_train_ = dataset.next_batch(100)
    sess.run(train_step, feed_dict={x: X_train_, y_: y_train_})

In [105]:
test_csv = pd.read_csv('./dataset/test.csv')
result = sess.run(y, feed_dict={x: test_csv})
result = [np.argmax(p) for p in result]
result = pd.DataFrame({'ImageId': range(1,len(result)+1), 'Label': result})
result.to_csv('result.csv', index=False, encoding='utf-8')

In [116]:
sess.close()